### LIBRERÍAS

In [6]:
# Librerías básicas
import numpy as np
import pandas as pd

# Librerías de Preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import metrics 


#Librerís de Modelado y evaluación
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pathlib import Path


### DATOS SÁTELITALES

In [7]:

satelitales = pd.read_csv('../Data/Boyaca_NDVI_EVI_SoilMoisture_MENSUAL_2005_2025_CLEAN.csv')
satelitales



,EVI,NDVI,fecha,year,month,yyyymm
0,0.381304,0.688825,2005-01-01,2005,1,2005-01
1,0.362783,0.668144,2005-02-01,2005,2,2005-02
2,0.379677,0.632942,2005-03-01,2005,3,2005-03
3,0.435677,0.667824,2005-04-01,2005,4,2005-04
4,0.405432,0.695091,2005-05-01,2005,5,2005-05
...,...,...,...,...,...,...
243,0.419982,0.695208,2025-04-01,2025,4,2025-04
244,0.434576,0.731797,2025-05-01,2025,5,2025-05
245,0.405199,0.743507,2025-06-01,2025,6,2025-06
246,0.453621,0.732904,2025-07-01,2025,7,2025-07


In [8]:
satelitales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   EVI     248 non-null    float64
 1   NDVI    248 non-null    float64
 2   fecha   248 non-null    object 
 3   year    248 non-null    int64  
 4   month   248 non-null    int64  
 5   yyyymm  248 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 11.8+ KB


In [9]:
satelitales.describe()

,EVI,NDVI,year,month
count,248.000000,248.000000,248.000000,248.000000
mean,0.415884,0.690760,2014.838710,6.435484
std,0.030513,0.031170,5.980247,3.445920
min,0.322249,0.547600,2005.000000,1.000000
25%,0.396819,0.674575,2010.000000,3.000000
50%,0.419184,0.697605,2015.000000,6.000000
75%,0.437864,0.712912,2020.000000,9.000000
max,0.499423,0.746885,2025.000000,12.000000


### ACOTACIÓN TIEMPO Y CORRECIÓN FECHA

In [10]:


# 1) Copia y normaliza nombres
df_ndvi = satelitales.copy()
df_ndvi.columns = df_ndvi.columns.str.strip().str.lower()

# 2) Detecta columna de fecha y pásala a datetime
fecha_col = 'fecha' if 'fecha' in df_ndvi.columns else [c for c in df_ndvi.columns if 'time' in c or 'fecha' in c][0]
df_ndvi[fecha_col] = pd.to_datetime(df_ndvi[fecha_col], errors='coerce')

# 3) Filtra 2012–2019
df_ndvi = df_ndvi[(df_ndvi[fecha_col].dt.year >= 2012) & (df_ndvi[fecha_col].dt.year <= 2019)]

# 4) Quédate solo con fecha + NDVI/EVI (en cualquier variante de nombre)
idx_cols = [c for c in df_ndvi.columns if c == 'ndvi' or c == 'evi' or c.startswith('ndvi') or c.startswith('evi')]
df_ndvi = df_ndvi[[fecha_col] + idx_cols].sort_values(fecha_col).reset_index(drop=True)
# (Opcional) Si hubiera filas repetidas por fecha:
# df_ndvi = df_ndvi.drop_duplicates(subset=[fecha_col])

# 5) Campos auxiliares
df_ndvi['year'] = df_ndvi[fecha_col].dt.year
df_ndvi['month'] = df_ndvi[fecha_col].dt.month
df_ndvi['yyyymm'] = df_ndvi[fecha_col].dt.strftime('%Y-%m')

print("NDVI/EVI listo:", df_ndvi.shape)
display(df_ndvi.head())


NDVI/EVI listo: (96, 6)


,fecha,evi,ndvi,year,month,yyyymm
0,2012-01-01,0.398118,0.699554,2012,1,2012-01
1,2012-02-01,0.390345,0.671628,2012,2,2012-02
2,2012-03-01,0.376592,0.643635,2012,3,2012-03
3,2012-04-01,0.451125,0.697795,2012,4,2012-04
4,2012-05-01,0.447870,0.697231,2012,5,2012-05


### RENDIMIENTOS CULTIVO CAFÉ

In [11]:
rendimientos = pd.read_excel('../Data/Producción y Rendimiento Mensual en Boyaca_12-19.xlsx')
rendimientos


,Fecha,Año,Mes,Boyaca_Prod_toneladas,AreaCultivada_ha,Rendimiento_t-ha
0,2012-01-01,2012,1,91.463307,4791.05,0.027630
1,2012-02-01,2012,2,97.617847,4791.05,0.029489
2,2012-03-01,2012,3,98.472645,4791.05,0.029747
3,2012-04-01,2012,4,99.156482,4791.05,0.029954
4,2012-05-01,2012,5,117.791063,4791.05,0.035583
...,...,...,...,...,...,...
91,2019-08-01,2019,8,748.984354,12312.95,0.067459
92,2019-09-01,2019,9,728.235011,12312.95,0.065591
93,2019-10-01,2019,10,916.317766,12312.95,0.082531
94,2019-11-01,2019,11,1008.016476,12312.95,0.090790


In [12]:
rendimientos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha                  96 non-null     datetime64[ns]
 1   Año                    96 non-null     int64         
 2   Mes                    96 non-null     int64         
 3   Boyaca_Prod_toneladas  96 non-null     float64       
 4   AreaCultivada_ha       96 non-null     float64       
 5   Rendimiento_t-ha       96 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 4.6 KB


In [13]:
rendimientos.describe()

,Fecha,Año,Mes,Boyaca_Prod_toneladas,AreaCultivada_ha,Rendimiento_t-ha
count,96,96.000000,96.000000,96.000000,96.000000,96.000000
mean,2015-12-16 11:00:00,2015.500000,6.500000,471.616979,7602.197500,0.067611
min,2012-01-01 00:00:00,2012.000000,1.000000,88.727956,4232.810000,0.026803
25%,2013-12-24 06:00:00,2013.750000,3.750000,219.163346,5139.477500,0.053199
50%,2015-12-16 12:00:00,2015.500000,6.500000,456.275407,6723.035000,0.068274
75%,2017-12-08 18:00:00,2017.250000,9.250000,721.758222,10195.470000,0.080680
max,2019-12-01 00:00:00,2019.000000,12.000000,1124.480531,12312.950000,0.115797
std,NaN,2.303316,3.470174,274.341289,2861.117089,0.020592


### DATOS ANUALES - CONVERSIÓN A MENSUALES

In [ ]:
# ============================================
# RENDIMIENTOS REALES DESDE EXCEL (BOYACÁ)
# ============================================

# 1) Partimos del DataFrame `rendimientos` leído del Excel
#   Columnas reales del archivo:
#   ['Fecha', 'Año', 'Mes', 'Boyaca_Prod_toneladas',
#    'AreaCultivada_ha', 'Rendimiento_t-ha']

# Renombrar columnas para que coincidan con las usadas en el notebook viejo
rend = rendimientos.rename(columns={
    "Fecha": "fecha",
    "Año": "anio",
    "Mes": "month",
    "Rendimiento_t-ha": "rendimiento_t_ha"
}).copy()

# 2) Campos adicionales que antes se fabricaban
rend["departamento"] = "Boyacá"
rend["fecha"] = pd.to_datetime(rend["fecha"], errors="coerce")
rend["yyyymm"] = rend["fecha"].dt.strftime("%Y-%m")

# 3) Este es el DataFrame que antes salía del código simulado
df_rend_boyaca = rend[[
    "departamento",
    "fecha",
    "anio",
    "month",
    "yyyymm",
    "rendimiento_t_ha"
]].copy()

# 4) Comprobación rápida de que cuadra por año
df_check = df_rend_boyaca.groupby("anio")["rendimiento_t_ha"].sum().round(3)
print("🔎 Comprobación de sumas anuales (t/ha):")
print(df_check)

print("\n👀 Primeras filas de df_rend_boyaca:")
display(df_rend_boyaca.head(24))




🔎 Comprobación de sumas anuales (t/ha):
anio
2012    0.400
2013    0.862
2014    0.722
2015    0.599
2016    0.986
2017    1.060
2018    0.972
2019    0.889
Name: rendimiento_t_ha, dtype: float64

👀 Primeras filas de df_rend_boyaca:


,departamento,fecha,anio,month,yyyymm,rendimiento_t_ha
0,Boyacá,2012-01-01,2012,1,2012-01,0.027630
1,Boyacá,2012-02-01,2012,2,2012-02,0.029489
2,Boyacá,2012-03-01,2012,3,2012-03,0.029747
3,Boyacá,2012-04-01,2012,4,2012-04,0.029954
4,Boyacá,2012-05-01,2012,5,2012-05,0.035583
5,Boyacá,2012-06-01,2012,6,2012-06,0.036874
6,Boyacá,2012-07-01,2012,7,2012-07,0.034498
7,Boyacá,2012-08-01,2012,8,2012-08,0.029179
8,Boyacá,2012-09-01,2012,9,2012-09,0.026803
9,Boyacá,2012-10-01,2012,10,2012-10,0.033724



Cada año se descompone en 12 meses, respetando la estacionalidad típica del café:

Mitaca: abril–junio con picos leves.

Principal: octubre–diciembre con picos fuertes.

Enero–marzo y julio–septiembre con actividad moderada/baja.

### SIMULACIÓN VARIABLES CLIMÁTICAS

In [15]:
import pandas as pd
import numpy as np

# ============================================================
# 0. Rutas de archivos (ajusta si tus carpetas cambian)
# ============================================================
FILE_CLIMA_BASE = "../Data/clima_boyaca_temp_hum_2012_2019.csv"
FILE_PRECIP     = "../Data/precipitacion_boy.csv"
FILE_SALIDA     = "../Data/clima_boyaca_2012_2019_completo.csv"

# ============================================================
# 1. Cargar clima base (TempMax, TempMin, Humedad) YA LIMPIO
# ============================================================
df_clima_base = pd.read_csv(FILE_CLIMA_BASE)
df_clima_base.columns = df_clima_base.columns.str.strip()

# Asegurar que 'fecha' es datetime
df_clima_base["fecha"] = pd.to_datetime(df_clima_base["fecha"], errors="coerce")

print("Clima base (TEMP + HUM):", df_clima_base.shape)


# Debe tener columnas: fecha, TempMax, TempMin, HumedadRelativa
# (si cambian de nombre, revisa aquí)
assert "fecha" in df_clima_base.columns, "El CSV base debe tener columna 'fecha'"
assert "TempMax" in df_clima_base.columns, "Falta columna 'TempMax' en clima base"
assert "TempMin" in df_clima_base.columns, "Falta columna 'TempMin' en clima base"
assert "HumedadRelativa" in df_clima_base.columns, "Falta 'HumedadRelativa' en clima base"


# ============================================================
# 2. Cargar PRECIPITACIÓN mensual (día = mes)
# ============================================================
df_p = pd.read_csv(FILE_PRECIP)
df_p.columns = df_p.columns.str.strip()

# Tomamos las dos columnas (Fecha, P_promedio_mm)
col_fecha_p = df_p.columns[0]  # debería ser "Fecha"
col_pp      = df_p.columns[1]  # debería ser "P_promedio_mm"

df_p[col_fecha_p] = pd.to_datetime(df_p[col_fecha_p], errors="coerce")

# AÑO correcto viene en la fecha
df_p["anio"] = df_p[col_fecha_p].dt.year

# El MES está codificado en el DÍA (1..12 = enero..diciembre)
df_p["mes"] = df_p[col_fecha_p].dt.day

# Construimos la fecha mensual real (1er día de cada mes)
df_p["fecha"] = pd.to_datetime(
    dict(year=df_p["anio"], month=df_p["mes"], day=1)
)

# Nos quedamos solo con fecha + precipitación
df_p_mensual = df_p[["fecha", col_pp]].copy()
df_p_mensual.rename(columns={col_pp: "Precipitacion"}, inplace=True)

print("Precipitación mensual:", df_p_mensual.shape)



# ============================================================
# 3. MERGE FINAL: clima base + precipitación
# ============================================================
df_clima = df_clima_base.merge(df_p_mensual, on="fecha", how="inner")

# Ordenar por fecha por prolijidad
df_clima = df_clima.sort_values("fecha").reset_index(drop=True)

print("df_clima FINAL (TempMax, TempMin, HumedadRelativa, Precipitacion):",
      df_clima.shape)
display(df_clima.head(12))



Clima base (TEMP + HUM): (96, 4)
Precipitación mensual: (96, 2)
df_clima FINAL (TempMax, TempMin, HumedadRelativa, Precipitacion): (96, 5)


,fecha,TempMax,TempMin,HumedadRelativa,Precipitacion
0,2012-01-01,23.042500,10.575833,85.090833,89.925
1,2012-02-01,23.779167,10.154167,80.866667,64.750
2,2012-03-01,23.403333,12.153333,85.336667,224.175
3,2012-04-01,22.692500,12.070833,88.565833,272.475
4,2012-05-01,22.463333,11.777500,87.864167,204.925
5,2012-06-01,22.598333,10.580833,85.214167,134.600
6,2012-07-01,21.725000,10.990833,86.235000,164.600
7,2012-08-01,22.355000,10.965833,85.379167,180.350
8,2012-09-01,22.710833,11.063333,82.595000,82.925
9,2012-10-01,23.313333,11.260833,86.170833,201.500


### INTEGRACIÓN DATAFRAMES

In [16]:
#INTEGRACIÓN 

# yyyymm de cada fuente
df_ndvi["yyyymm"]= df_ndvi[fecha_col].dt.strftime("%Y-%m")
df_rend_boyaca["yyyymm"]= df_rend_boyaca["fecha"].dt.strftime("%Y-%m")
df_clima["yyyymm"]      = df_clima["fecha"].dt.strftime("%Y-%m")

# Merge por yyyymm
df_modelo = (
    df_ndvi.merge(df_rend_boyaca[["yyyymm","rendimiento_t_ha"]], on="yyyymm", how="inner")
           .merge(df_clima.drop(columns=["fecha"], errors="ignore"), on="yyyymm", how="left")
           .rename(columns={"rendimiento_t_ha":"rendimiento"})
           .assign(fecha=lambda d: pd.to_datetime(d["yyyymm"]))
           .assign(anio=lambda d: d["fecha"].dt.year, month=lambda d: d["fecha"].dt.month)
           .sort_values("fecha", kind="stable")
           .reset_index(drop=True)
)

# (opcional) Reordenar para visualización rápida
cols = ["fecha","yyyymm","ndvi", "evi", "rendimiento","Precipitacion","TempMax","TempMin","HumedadRelativa"]
df_modelo = df_modelo[[c for c in cols if c in df_modelo.columns]]

# Mostrar tabla limpia
print("Dataset mensual para modelado:", df_modelo.shape)
display(df_modelo.head(12))

Dataset mensual para modelado: (96, 9)


,fecha,yyyymm,ndvi,evi,rendimiento,Precipitacion,TempMax,TempMin,HumedadRelativa
0,2012-01-01,2012-01,0.699554,0.398118,0.027630,89.925,23.042500,10.575833,85.090833
1,2012-02-01,2012-02,0.671628,0.390345,0.029489,64.750,23.779167,10.154167,80.866667
2,2012-03-01,2012-03,0.643635,0.376592,0.029747,224.175,23.403333,12.153333,85.336667
3,2012-04-01,2012-04,0.697795,0.451125,0.029954,272.475,22.692500,12.070833,88.565833
4,2012-05-01,2012-05,0.697231,0.447870,0.035583,204.925,22.463333,11.777500,87.864167
5,2012-06-01,2012-06,0.710103,0.415739,0.036874,134.600,22.598333,10.580833,85.214167
6,2012-07-01,2012-07,0.704582,0.441602,0.034498,164.600,21.725000,10.990833,86.235000
7,2012-08-01,2012-08,0.700551,0.423252,0.029179,180.350,22.355000,10.965833,85.379167
8,2012-09-01,2012-09,0.691045,0.397087,0.026803,82.925,22.710833,11.063333,82.595000
9,2012-10-01,2012-10,0.702355,0.421132,0.033724,201.500,23.313333,11.260833,86.170833


### PREPROCESAMIENTO

In [17]:
# PREPROCESAMIENTO SIMPLE

# 1) Copia y asegura columnas de tiempo
datos = df_modelo.copy()

# fecha ↔ yyyymm
if "fecha" not in datos.columns:
    if "yyyymm" in datos.columns:
        datos["fecha"] = pd.to_datetime(datos["yyyymm"], format="%Y-%m", errors="coerce")
    else:
        raise ValueError("Falta 'fecha' o 'yyyymm' en df_modelo.")
datos["mes"]  = datos["fecha"].dt.month
datos["anio"] = datos["fecha"].dt.year

# 2) Estandariza nombres NDVI/EVI y clima (tolerante a mayúsculas/minúsculas)
renombres = {}
for c in datos.columns:
    cl = c.strip().lower()
    if cl.startswith("ndvi"): renombres[c] = "NDVI"
    if cl.startswith("evi"):  renombres[c] = "EVI"
    if cl in {"precipitacion"}: renombres[c] = "Precipitacion"
    if cl in {"tmax","tempmax"}: renombres[c] = "TempMax"
    if cl in {"tmin","tempmin"}: renombres[c] = "TempMin"
    if cl in {"humedad","humedadrelativa"}: renombres[c] = "HumedadRelativa"
datos = datos.rename(columns=renombres)

# 3) Estacionalidad mensual (seno/coseno)
datos["mes_sin"] = np.sin(2*np.pi*datos["mes"]/12)
datos["mes_cos"] = np.cos(2*np.pi*datos["mes"]/12)

# 4) Selección de variables (toma solo las que existan)
variables_base = ["NDVI","EVI","Precipitacion","TempMax","TempMin","HumedadRelativa"]
variables_modelo = [v for v in variables_base if v in datos.columns] + ["mes_sin","mes_cos"]

X = datos[variables_modelo].apply(pd.to_numeric, errors="coerce")   # predictores
y = pd.to_numeric(datos["rendimiento"], errors="coerce")            # objetivo

# 5) División temporal y transformación (imputar + escalar)
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(
    X, y, test_size=0.20, shuffle=False
)

prepro = Pipeline([
    ("imputar", SimpleImputer(strategy="median")),
    ("escalar", StandardScaler())
])

X_entrena_prep = prepro.fit_transform(X_entrena)
X_prueba_prep  = prepro.transform(X_prueba)

print("Variables usadas:", variables_modelo)
print("Variable objetivo:", "rendimiento")
print("Train:", X_entrena_prep.shape, " | Test:", X_prueba_prep.shape)


Variables usadas: ['NDVI', 'EVI', 'Precipitacion', 'TempMax', 'TempMin', 'HumedadRelativa', 'mes_sin', 'mes_cos']
Variable objetivo: rendimiento
Train: (76, 8)  | Test: (20, 8)


## 🌎 ¿Por qué usamos `mes_sin` y `mes_cos`?

En datos **mensuales** como los de la caficultura, el **mes del año influye fuertemente** en el rendimiento (por lluvias, floración, cosecha, etc.).  
Sin embargo, el número del mes (`1–12`) es una **variable cíclica**, no lineal.

### 🧩 Problema
Si usas el número del mes directamente, el modelo interpretará que:

> Diciembre (12) está muy lejos de Enero (1),

cuando en realidad **son adyacentes** en el ciclo anual.

### 🧭 Solución: *codificación cíclica*

Transformamos el número del mes en **coordenadas circulares** usando funciones seno y coseno.  
Así, los meses se representan sobre un círculo, donde diciembre y enero “se tocan”.

### 📐 Fórmulas

\[
\text{mes\_sin} = \sin\left(2\pi \times \frac{\text{mes}}{12}\right)
\]

\[
\text{mes\_cos} = \cos\left(2\pi \times \frac{\text{mes}}{12}\right)
\]

### 🌀 Ejemplos

| Mes | mes_sin | mes_cos | Interpretación |
|-----|----------|----------|----------------|
| Enero | ≈ 0.5 | ≈ 0.87 | Inicio de año |
| Julio | ≈ -1 | ≈ 0 | Mitad del año |
| Diciembre | ≈ 0 | ≈ 1 | Fin del año |

### 💡 Beneficio

El modelo aprende **la estacionalidad continua**, es decir, los cambios **graduales entre meses**,  
sin los saltos artificiales que ocurren si se usan números enteros.


### MODELADO Y SIMULACIÓN

In [18]:
#  MODELADO Y EVALUACIÓN 

# Modelos a probar
modelos = {
    "Regresión Lineal": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=21),
    "Gradient Boosting": GradientBoostingRegressor(random_state=21)
}

resultados = []

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos.items():
    modelo.fit(X_entrena_prep, y_entrena)
    y_pred = modelo.predict(X_prueba_prep)
    
    mae = mean_absolute_error(y_prueba, y_pred)
    rmse = np.sqrt(mean_squared_error(y_prueba, y_pred))
    r2 = r2_score(y_prueba, y_pred)
    
    resultados.append([nombre, mae, rmse, r2])

# Tabla de resultados
df_resultados = pd.DataFrame(resultados, columns=["Modelo", "MAE", "RMSE", "R2"]).sort_values("R2", ascending=False)

print("📊 Resultados del modelado:")
display(df_resultados)

# Mejor modelo
mejor_modelo = df_resultados.iloc[0, 0]
print(f"✅ Mejor modelo: {mejor_modelo}")


📊 Resultados del modelado:


,Modelo,MAE,RMSE,R2
0,Regresión Lineal,0.011549,0.013836,-0.429031
1,Random Forest,0.010745,0.013964,-0.455640
2,Gradient Boosting,0.014089,0.018384,-1.523098


✅ Mejor modelo: Regresión Lineal


### EXPORTO DF_MODELO PARA PODER USAR EN PIPELINE

In [19]:
from pathlib import Path

CACHE = Path("artifacts/_cache")
CACHE.mkdir(parents=True, exist_ok=True)

# Formato preferido (conserva tipos y es más compacto)
df_modelo.to_parquet(CACHE / "df_modelo.parquet", index=False)

# Opcional: también como CSV por si quieres verlo a ojo
df_modelo.to_csv(CACHE / "df_modelo.csv", index=False, encoding="utf-8-sig")

print("✅ Exportado df_modelo a artifacts/_cache/")


✅ Exportado df_modelo a artifacts/_cache/
